In [14]:
import os
import json
import numpy as np
import pandas as pd
import networkx as nx
import keplergl
from helpers.helperFunctions import readJSONDiGraph, lineElevationProfile, euclideanDistance, writeJSONFile
from shapely.geometry import Point, MultiPoint
from path_calculation import calculate_cost

In [2]:
filename = "data/roadNetwork-Directed-TokyoArea-primary-v6.json"
G = readJSONDiGraph(filename)

In [3]:
alpha = 1.5
factor = 2 
nodes = pd.DataFrame([{'id': node[0], **node[1]} for node in list(G.nodes(data=True))])
G = calculate_cost(G, alpha=alpha, factor=factor)

In [11]:
unG = G.to_undirected()

In [12]:
sourceId = 517547602
targetId = 1681663754
path = nx.dijkstra_path(unG, sourceId, targetId, weight='cost')

NetworkXNoPath: No path to 1681663754.

In [6]:
list(G.edges(data=True))[0]

(31236733,
 621545916,
 {'roadType': 'motorway',
  'roadName': '首都高速11号台場線',
  'oneWay': 1,
  'speedLimit': 80.0,
  'roadWidth': 21.0,
  'driveSpeed': 60,
  'capacity': 24000,
  'numLanes': 3,
  'modality': 'road',
  'x1': 139.7578,
  'y1': 35.64379520000001,
  'x2': 139.7578196,
  'y2': 35.6417602,
  'distance': 0.225795,
  'timeWeight': 0.000225795,
  'elevationGain': -0.4900000095367432,
  'cost': 642.371310228549})

In [8]:
links = pd.read_csv("data/filtered-elevationLinkData-TokyoArea-v6.csv")

/Users/s_araki/.pyenv/versions/3.6.5/envs/geoanalysis/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
kmap = keplergl.KeplerGl(height=400, data={"edges": links})
kmap.save_to_html(file_name = "test_size.html")

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to test_size.html!


In [15]:
largest_cc = max(nx.weakly_connected_components(G), key=len)

In [18]:
with open("data/roadNetwork-Directed-TokyoArea-with-cost-v6.json", encoding='utf-8-sig') as f:
        js_graph = json.load(f)

In [23]:
links = pd.DataFrame(js_graph['links'])
is_member = lambda row: row.source in largest_cc or row.target in largest_cc
primary_links = links.loc[links.apply(is_member, axis=1)]

In [24]:
primaryG = nx.from_pandas_edgelist(primary_links, 'source', 'target', True, nx.DiGraph())

In [25]:
writeJSONFile(primaryG, "data/roadNetwork-Directed-TokyoArea-primary-v6.json")

In [27]:
list(primaryG.nodes())[:10]

[31236733,
 621545916,
 621545917,
 31236732,
 1832774251,
 1832774248,
 31236568,
 31236567,
 31236566,
 1832774235]

In [28]:
list(primaryG.nodes(data=True))[0]

(31236733, {})

In [29]:
nodeData = pd.read_csv("data/filtered-nodeData-TokyoArea-v6.csv")

In [ ]:
for nodeID in list(primaryG.nodes()):
    primaryG.nodes[nodeID]['elevation'] = nodeData.loc[nodeData.id == nodeID].elevation.values[0]

In [ ]:
list(primaryG.nodes(data=True))[0]

In [ ]:
writeJSONFile(primaryG, "data/roadNetwork-Directed-TokyoArea-primary-v6.json")